# Worksheet 16

Name: Ryan Chung 
UID: U33101958

### Topics

- Support Vector Machines (Non-linear case)

## Support Vector Machines

Follow along in class to implement the perceptron algorithm and create an animation of the algorithm.

a) As we saw in class, the form
$$w^T x + b = 0$$
while simple, does not expose the inner product `<x_i, x_j>` which we know `w` depends on, having done the math. This is critical to applying the "kernel trick" which allows for learning non-linear decision boundaries. Let's modify the above algorithm to use the form
$$\sum_i \alpha_i <x_i, x> + b = 0$$

In [63]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
from tqdm import tqdm
import os

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

epochs = 100
learning_rate = .05
expanding_rate = .99
retracting_rate = 1.1

X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

alpha_i = np.zeros((len(X),))
b = 0

def snap(x, alpha_i, b, error):
    # create a mesh to plot in
    h = .01  # step size in the mesh
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    meshData = np.c_[xx.ravel(), yy.ravel()]
    cs = np.array([x for x in 'gb'])
    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)

    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='y',fill=False))
   
    Z = predict_many(alpha_i, b, meshData)
    Z = np.array([0 if z <=0 else 1 for z in Z]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=.5, cmap=plt.cm.Paired)
    fig.savefig(TEMPFILE)
    plt.close()
    return im.fromarray(np.asarray(im.open(TEMPFILE)))

def predict_many(alpha_i, b, Z):
    res = []
    for i in range(len(Z)):
        res.append(predict(alpha_i, b, Z[i]))
    return np.array(res)

def polynomial(x_i, x_j, c=1, d=2):
    return (np.dot(x_i, x_j) + c) ** d

def predict(alpha_i, b, x):
    wx = 0
    for j in range(len(X)):
        wx += alpha_i[j] * np.dot(X[j], x)
    return wx + b

images = []
for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    x, y = X[i], Y[i]
    ypred = predict(alpha_i, b, x)
    error = False 

    if (ypred > 0 and y > 0) or (ypred < 0 and y < 0):
        # classified correctly
        if ypred < 1 and ypred > -1:
            # in the street / street is too wide
            alpha_i[i] += y * learning_rate
            alpha_i = alpha_i * retracting_rate           
            b += y * learning_rate * retracting_rate
        else:
            # street is too narrow
            alpha_i = alpha_i * expanding_rate
            b *= expanding_rate
    else:
        # misclassified
        alpha_i[i] += y * learning_rate
        alpha_i = alpha_i * expanding_rate
        b += y * learning_rate * expanding_rate

    images.append(snap(x, alpha_i, b, error))

images[0].save(
    'svm_dual.gif',
    save_all=True,
    append_images=images[1:],
    optimize=False,
    duration=100,
    loop=0
)

Write a configurable kernel function to apply in lieu of the dot product. Try it out on a dataset that is not linearly separable.

In [ ]:
def polynomial(x_i, x_j, c, n):
    return (np.dot(x_i, x_j) + c) ** n

b) Assume we fit an SVM using a polynomial Kernel function and it seems to overfit the data. How would you adjust the tuning parameter `n` of the kernel function?

decrease n

c) Assume we fit an SVM using a RBF Kernel function and it seems to underfit the data. How would you adjust the tuning parameter `sigma` of the kernel function?

decrease sigma 

d) Tune the parameter of a specific Kernel function, to fit an SVM (using your code above) to the following dataset:

In [69]:
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt("spiral.data")
X, y = data[:, :2], data[:, 2]
Y = np.array([-1 if label == 0 else 1 for label in y])

epochs = 100
learning_rate = 0.05
expanding_rate = 0.99
retracting_rate = 1.1

alpha_i = np.zeros(len(X))
b = 0

def rbf_kernel(x_i, x_j, gamma=1):
    return np.exp(-gamma * np.linalg.norm(x_i - x_j) ** 2)

def predict(alpha_i, b, X, x):
    wx = 0
    for j in range(len(X)):
        wx += alpha_i[j] * Y[j] * rbf_kernel(X[j], x)
    return wx + b

for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    x, y = X[i], Y[i]
    ypred = predict(alpha_i, b, X, x)

    if (ypred > 0 and y > 0) or (ypred < 0 and y < 0):
        # classified correctly
        if abs(ypred) < 1:
            # in the street / street is too wide
            alpha_i[i] += y * learning_rate
            alpha_i *= retracting_rate
            b += y * learning_rate * retracting_rate
        else:
            # street is too narrow
            alpha_i *= expanding_rate
            b *= expanding_rate
    else:
        # misclassified
        alpha_i[i] += y * learning_rate
        alpha_i *= expanding_rate
        b += y * learning_rate * expanding_rate

